In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [6]:
data=pd.read_csv("C:/Users/User/Desktop/assignment/generic-churn-prediction/data//Bank_churn.csv")

In [7]:
data.head()

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,churn
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
data.shape

(10000, 14)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           10000 non-null  int64  
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  churn            10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [12]:
data.describe()

,rownumber,customerid,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,churn
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [26]:
data=data.drop('rownumber', axis=1)

In [27]:
# find categorical variables

cat_var=[col for col in data.columns if data[col].dtype =="object " or data[col].nunique()<=11]

In [28]:
cat_var

['geography',
 'gender',
 'tenure',
 'numofproducts',
 'hascrcard',
 'isactivemember',
 'churn']

In [29]:
numerical_var=[col for col in data.columns if data[col].dtype!= "object" and data[col].nunique()>11 and col not in "customerid"]

In [30]:
numerical_var

['creditscore', 'age', 'balance', 'estimatedsalary']

In [31]:
print(len(numerical_var), len(cat_var))

4 7


In [32]:
data["churn"].value_counts()

0    7963
1    2037
Name: churn, dtype: int64

In [33]:
data.isnull().sum()

customerid         0
surname            0
creditscore        0
geography          0
gender             0
age                0
tenure             0
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
churn              0
dtype: int64

In [36]:
data.fillna(method='ffill', inplace=True)
for col in data.select_dtypes(include='object').columns:
    data[col] = LabelEncoder().fit_transform(data[col])
scaler = StandardScaler()
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[num_cols] = scaler.fit_transform(data[num_cols])

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [43]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
if data['churn'].dtype == 'float64':
    data['churn'] = data['churn'].round().astype(int)

In [49]:
X = data.drop('churn', axis=1)
y = data['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = [('Log', LogisticRegression(random_state=42)),
          ('Knn', KNeighborsClassifier()),
          ('dec', DecisionTreeClassifier(random_state=42)),
          ('Ranf', RandomForestClassifier(random_state=42))]
cv_result = []
best_model=None
best_accuracy=0
for name, model in models:
    kfold = KFold(n_splits=10,  shuffle=True, random_state=42)
    cv_results = cross_val_score(model, X, y, cv=kfold)
    mean_accuracy = cv_results.mean()
    
    print(f"{name} Model Accuracy: {mean_accuracy:.4f}")
    
    # Check if this is the best model so far
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_model = model
    cv_result.append(cv_results)

Log Model Accuracy: 0.8073
Knn Model Accuracy: 0.7761
dec Model Accuracy: 0.7884
Ranf Model Accuracy: 0.8628


In [55]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [51]:
best_model

RandomForestClassifier(random_state=42)

In [52]:
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Best Model: {best_model.__class__.__name__} with accuracy: {test_accuracy:.4f}")

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

Best Model: RandomForestClassifier with accuracy: 0.8635


In [53]:
conf_matrix

array([[1553,   54],
       [ 219,  174]], dtype=int64)

In [56]:
print("True Positive : ", conf_matrix[1, 1])
print("True Negative : ", conf_matrix[0, 0])
print("False Positive: ", conf_matrix[0, 1])
print("False Negative: ", conf_matrix[1, 0])

True Positive :  174
True Negative :  1553
False Positive:  54
False Negative:  219


In [57]:
print(classification_report(best_model.predict(X_test),y_test))

              precision    recall  f1-score   support

          -1       0.97      0.88      0.92      1772
           2       0.44      0.76      0.56       228

    accuracy                           0.86      2000
   macro avg       0.70      0.82      0.74      2000
weighted avg       0.91      0.86      0.88      2000

